In [1]:
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as mc
import warnings
import numpy as np
from tqdm import tqdm
import spacy

In [2]:
def Add_subset(k, scores, test_idx, pred, X, y, thresh):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > thresh:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]
        
    y_k = y["train_add"][top_k_index]
    X_k = X["train_add"][top_k_index]

    prediction = -np.sum(scores[test_idx][top_k_index])
    #print("prediction", prediction)

    return X_k, y_k, prediction, top_k_index


def loss_gradient(X, y, model):
    F_train = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
    error_train = model.predict_proba(X)[:, 1] - y
    gradient_train = F_train * error_train[:, None]
    return gradient_train

In [3]:
def approximate_k(test_idx, pred, delta_pred, y, thresh):
    old = pred[test_idx].item()
    
    if pred[test_idx] > thresh:
        top_k_index = np.flip(delta_pred[test_idx].argsort())
    else:
        top_k_index = delta_pred[test_idx].argsort()
    
    for k in range(1, y["train_add"].shape[0]):
        change = np.sum(delta_pred[test_idx][top_k_index[:k]])
        
        if old > thresh and old + change < thresh:
            return k
        elif old < thresh and old + change > thresh:
            return k
        
    return None

In [4]:
def new_train(k, dev_index, scores, l2, X, model, pred, y, thresh):
    X_k, y_k, prediction, top_k_index = Add_subset(k, scores, dev_index, pred, X, y, thresh)
    
    if y_k.shape[0] == np.sum(y_k) or np.sum(y_k) == 0: # data contains only one class: 1
        return None, None, None, None, None, None
    
    model.fit(X_k, y_k)

    # predictthe probaility with test point
    test_point = X["dev"][dev_index]
    test_point=np.reshape(test_point, (1,-1))
    
    new = model.predict_proba(test_point)[0][1]
    change = -(model.predict_proba(test_point)[0][1] - new)
    #change = model_k.predict_proba(test_point)[0][1]-model.predict_proba(test_point)[0][1]
    flip = (model.predict(test_point) == model.predict(test_point))
    
    """
    print("change    ", change)
    print("old       ", model.predict_proba(test_point)[0][1])
    print()
    """
    error = np.abs((change - prediction)/prediction)
    return change, flip, prediction, new, error, top_k_index

In [5]:
def IP(X, y, l2, dataname, thresh, model, pred, modi=None):
    gradient_train_add = loss_gradient(X["train_add"], y["train_add"], model)

    w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
    F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))],
                             axis=1)  # Concatenating one to calculate the gradient with respect to intercept
    F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

    # error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
    # error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

    # gradient_train = F_train * error_train[:, None]
    # gradient_dev = F_dev * error_dev[:, None]

    probs = model.predict_proba(X["train"])[:, 1]
    hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / \
              X["train"].shape[0]
    inverse_hessian = np.linalg.inv(hessian)
    eps = 1 / X["train"].shape[0]
    delta_k = -eps * inverse_hessian @ (gradient_train_add).T
    grad_f = F_dev * (pred * (1 - pred))
    delta_pred = grad_f @ delta_k
    return delta_pred
    


    


In [6]:
import spacy
sst_dataset = {}
for split in ["train", "dev", "test"]:
    URL = f"https://raw.githubusercontent.com/successar/instance_attributions_NLP/master/Datasets/SST/data/{split}.jsonl"
    import urllib.request, json
    with urllib.request.urlopen(URL) as url:
        data = url.read().decode()
        data = [json.loads(line) for line in data.strip().split("\n")]
        sst_dataset[split] = data


nlp = spacy.load('en_core_web_sm')

X, y = {}, {}
for split in ["train", "dev"]:
    X[split] = np.array([nlp(example["document"]).vector for example in tqdm(sst_dataset[split])])
    y[split] = np.array([example["label"] for example in sst_dataset[split]])

100%|██████████| 872/872 [00:05<00:00, 161.04it/s]


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_train_add, y_train, y_train_add = train_test_split(
    X["train"], y["train"], test_size=0.4, random_state=42)

In [8]:
X["train"] = X_train
X["train_add"] = X_train_add
y["train"] = y_train
y["train_add"]= y_train_add
X["train"].shape, X["train_add"].shape, X["dev"].shape

((4152, 96), (2768, 96), (872, 96))

In [12]:
thresh = 0.5
l2 = 10
model = LogisticRegression(penalty='l2', C=1/l2, solver='saga',warm_start=True)
model.fit(X["train"], y["train"])
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
np.sum(model.predict(X["dev"]) == y["dev"]) / 872

np.float64(0.6169724770642202)

In [13]:
delta_pred = IP(X, y, l2, "SST", thresh, model, pred)
delta_pred.shape

(872, 2768)

In [11]:
# # Loop over all dev points for evaluation:
# appro_ks = []
# new_predictions = []
# flip_list = []
# for test_idx in tqdm(range(X["dev"].shape[0])):
#     appro_k = approximate_k(test_idx, pred, delta_pred, y, thresh)
#     if appro_k != None:
#         change, _, prediction,new_prediction, error, top_k_index = new_train(appro_k, test_idx, delta_pred, l2, X, model, pred, y, thresh)
#         print(test_idx, appro_k, pred[test_idx], new_prediction)
#         appro_ks.append(appro_k)
#         new_predictions.append(new_prediction)
#         flip_list.append(top_k_index)
#     else:
#         appro_ks.append(None)
#         new_predictions.append(None)
#         flip_list.append(None)

# appro_ks= np.array(appro_ks)
# new_predictions=np.array(new_predictions)
# # flip_list = np.array(flip_list)
# # np.save("./results/" + "appro_ks_IP" + "_alg1_" + dataname  + str(l2) + ".npy", appro_ks)
# # np.save("./results/" + "new_predictions" + "_alg1_" + dataname + str(l2) + ".npy", new_predictions)
# # np.save("./results/" + "old_predictions" + "_alg1_" + dataname + str(l2) + ".npy", pred)
# # np.save("./results/" + "flip_list" + "_alg1_" + dataname + str(l2) + ".npy", flip_list)

  1%|          | 10/872 [00:00<00:19, 44.02it/s]

3 2761 [0.49117133] 0.52867776


  5%|▌         | 46/872 [00:00<00:17, 48.43it/s]

38 2766 [0.487544] 0.5140793


 10%|█         | 88/872 [00:01<00:16, 48.20it/s]

...

847 2759 [0.50584453] 0.47571823
854 2766 [0.46865052] 0.53216314


 99%|█████████▉| 863/872 [00:20<00:00, 35.49it/s]

857 2751 [0.5154432] 0.52937716
860 2764 [0.453544] 0.53188616


 99%|█████████▉| 867/872 [00:20<00:00, 42.85it/s]


KeyboardInterrupt: 

In [14]:
# new_predictions = new_predictions.reshape((872, 1))
# eval = []
# num = 0
# for i in range(872):
#     if new_predictions[i] != None:
#         eval.append((new_predictions[i] > 0.5) != (pred[i] > 0.5))
#         num += 1
# np.sum(eval) / num

# Accuracy Algorithm1

In [15]:
delta_pred.shape, delta_pred[:, 0].shape, pred.shape, y["dev"].shape

((872, 2768), (872,), (872, 1), (872,))

In [16]:
original_acc = np.sum((pred[:, 0] > 0.5) == y["dev"])/ y["dev"].shape[0]
original_acc, model.score(X["dev"], y["dev"])

(np.float64(0.6169724770642202), 0.6169724770642202)

In [17]:
def accuracy_higher(delta_pred, pred, y_truth):
    accmul = 0
    for i in range(delta_pred.shape[0]):
        # if goes same way
        # print(y_truth[i], pred[i][0], delta_pred[i], accmul)
        if np.sign(y_truth[i] - pred[i]) == np.sign(delta_pred[i]):
            accmul = accmul + np.abs(delta_pred[i])
        else:
            accmul = accmul - np.abs(delta_pred[i])
    if accmul > 0:
        return True
    else:
        return False



In [18]:
add_idx = []
for j in range(delta_pred.shape[1]):
    improve = accuracy_higher(delta_pred[:, j], pred, y["dev"])
    if improve == True:
        add_idx.append(j)
len(add_idx)

1674

In [ ]:
# continue to train with selected data
y_k = y["train_add"][add_idx]
X_k = X["train_add"][add_idx]
model.fit(X_k, y_k)
model.score(X["dev"], y["dev"])

0.6444954128440367

In [19]:
# continue to train with all data
y_k = y["train_add"]
X_k = X["train_add"]
model.fit(X_k, y_k)
model.score(X["dev"], y["dev"])

0.6295871559633027